In [2]:
class Cell:
    WHITE = 6
    ZERO = 0
    ONE = 1
    TWO = 2
    THREE = 3
    FOUR = 4
    ANY = 5
    LAMP = 7
    LIT = 8
    COLLISION = 9
    FORBIDDEN = 10
    AVAILABLE = 'AVAILABLE'
    NOT_AVAILABLE = 'NOT_AVAILABLE'
    BULB_PLACED = 'BULB_PLACED'

    def __init__(self, number) -> None:
        self.type = number
        self.available = [None, None, None, None]
        self.countAvailable = self.type
        
    def setAvailable (self, available):
        self.available = available
        self.checkAvailable()
    
    def setType (self, newType):
        self.type = newType

    def checkAvailable (self):
        self.countAvailable = self.type
        for i in range (4):
            if self.available[i] == Cell.BULB_PLACED:
                self.countAvailable -= 1
        if self.countAvailable == 0: return False
        else: return True

In [3]:

import numpy as np
import copy

class Board:
    def __init__(self, firstBoard) -> None:
        self.makeBoard(firstBoard)
        self.movement = [[0, 1, 0, -1], [1, 0, -1, 0]]

    def makeBoard (self, firstBoard):
        board = []
        self.rowSize = len(firstBoard)
        self.colSize = len(firstBoard[0])
        for row in firstBoard:
            col = []
            for element in row:
                col.append(Cell(element))
            board.append(col)
        self.board = np.array(board)

    def printBoard (self):
        for row in self.board:
            for element in row:
                p = ''
                e = element.type
                if e == Cell.ZERO: p = '0'
                elif e == Cell.ONE: p = '1'
                elif e == Cell.TWO: p = '2'
                elif e == Cell.THREE: p = '3'
                elif e == Cell.FOUR: p = '4'
                elif e == Cell.ANY: p = 'A'
                elif e == Cell.WHITE: p = 'W'
                elif e == Cell.LAMP: p = 'B'
                elif e == Cell.LIT: p = 'L'
                elif e == Cell.COLLISION: p = 'C'
                elif e == Cell.FORBIDDEN: p = 'F'
                print (p, end=' ')
            print()
        print()
    
    def printAvailable (self):
        for row in self.board:
            for element in row:
                if element.type <= Cell.ANY:
                    print (element.type, element.available, element.checkAvailable(), element.countAvailable)

    def setForbidden (self):
        for i, row in enumerate (self.board):
                for j, element in enumerate (row):
                    if element.type == Cell.ZERO:
                        for k in range (4):
                            nextRow = i + self.movement[0][k]
                            nextCol = j + self.movement[1][k]
                            if (nextRow >= 0 and nextCol >= 0 and nextRow < self.rowSize and nextCol < self.colSize):
                                if self.board[nextRow][nextCol].type == Cell.WHITE: 
                                    self.board[nextRow][nextCol].setType(Cell.FORBIDDEN)
                    elif element.type <= Cell.FOUR and not element.checkAvailable():
                        for k in range (4):
                            nextRow = i + self.movement[0][k]
                            nextCol = j + self.movement[1][k]
                            if (nextRow >= 0 and nextCol >= 0 and nextRow < self.rowSize and nextCol < self.colSize):
                                if element.available[k] == Cell.AVAILABLE and self.board[nextRow][nextCol].type != Cell.LIT:
                                    self.board[nextRow][nextCol].setType(Cell.FORBIDDEN)

    def setAvailability (self):
        for i, row in enumerate (self.board):
                for j, element in enumerate (row):
                    if (element.type < Cell.ANY and element.type >= Cell.ZERO):
                        available = []
                        for k in range (4):
                            nextRow = i + self.movement[0][k]
                            nextCol = j + self.movement[1][k]
                            if (nextRow >= 0 and nextCol >= 0 and nextRow < self.rowSize and nextCol < self.colSize):
                                if self.board[nextRow][nextCol].type == Cell.WHITE: available.append(Cell.AVAILABLE)
                                elif self.board[nextRow][nextCol].type == Cell.LAMP: available.append(Cell.BULB_PLACED)
                                else: available.append(Cell.NOT_AVAILABLE)
                            else: available.append(Cell.NOT_AVAILABLE)
                        element.setAvailable(available)
    def preproc (self):
        while True:
            check = False
            for i, row in enumerate (self.board):
                for j, element in enumerate (row):
                    if (element.type <= Cell.FOUR and element.type > Cell.ZERO and element.checkAvailable() == True):
                        countWhite = 0
                        whitePosition = []
                        for k in range (4):
                            nextRow = i + self.movement[0][k]
                            nextCol = j + self.movement[1][k]
                            if (nextRow >= 0 and nextCol >= 0 and nextRow < self.rowSize and nextCol < self.colSize):
                                if self.board[nextRow][nextCol].type == Cell.WHITE:
                                    countWhite += 1
                                    whitePosition.append([nextRow, nextCol])
                        if countWhite == element.countAvailable:
                            check = True
                            for white in whitePosition: 
                                self.putSingleLamp(white[0], white[1])
                                self.setForbidden()
                                self.setAvailability()
                    elif (element.type == Cell.WHITE):
                        checkWhiteCenter = True
                        for k in range (4):
                            nextRow = i + self.movement[0][k]
                            nextCol = j + self.movement[1][k]
                            if (nextRow >= 0 and nextCol >= 0 and nextRow < self.rowSize and nextCol < self.colSize):
                                if self.board[nextRow][nextCol].type > Cell.ANY:
                                    checkWhiteCenter = False
                                    break
                        if checkWhiteCenter: 
                            self.putSingleLamp(i, j)
                            self.setForbidden()
                            self.setAvailability()
                            check = True
            if not check: break

    def preprocSecond(self):
        while True:
            check = False
            for i, row in enumerate (self.board):
                for j, cell in enumerate (row):
                    if (cell.type <= Cell.FOUR and cell.type > Cell.ZERO and cell.checkAvailable() == True):
                        countWhite = 0
                        whitePosition = []
                        for k in range (4):
                            nextRow = i + self.movement[0][k]
                            nextCol = j + self.movement[1][k]
                            if (nextRow >= 0 and nextCol >= 0 and nextRow < self.rowSize and nextCol < self.colSize):
                                if self.board[nextRow][nextCol].type == Cell.WHITE:
                                    countWhite += 1
                                    whitePosition.append([nextRow, nextCol])
                        if countWhite == cell.countAvailable:
                            check = True
                            for white in whitePosition: 
                                self.putSingleLamp(white[0], white[1])
                                self.setForbidden()
                                self.setAvailability()
                    elif (cell.type == Cell.FORBIDDEN or cell.type == Cell.WHITE):
                        rowPos = i
                        colPos = j
                        countWhite = 0
                        resRow = None
                        resCol = None
                        # check rightside
                        for k in range (colPos+1, self.colSize):
                            if (k < 0 or k >= self.colSize or self.board[rowPos][k].type <= Cell.ANY): break
                            if (self.board[rowPos][k].type == Cell.WHITE):
                                resRow = rowPos
                                resCol = k
                                countWhite += 1
                        # check leftside
                        for k in range (colPos-1, -1, -1):
                            if (k < 0 or k >= self.colSize or self.board[rowPos][k].type <= Cell.ANY): break
                            if (self.board[rowPos][k].type == Cell.WHITE):
                                resRow = rowPos
                                resCol = k
                                countWhite += 1
                        # check downside
                        for k in range (rowPos+1, self.rowSize):
                            if (k < 0 or k >= self.rowSize or self.board[k][colPos].type <= Cell.ANY): break
                            if (self.board[k][colPos].type == Cell.WHITE):
                                resRow = k
                                resCol = colPos
                                countWhite += 1
                        # check upside
                        for k in range (rowPos-1, -1, -1):
                            if (k < 0 or k >= self.rowSize or self.board[k][colPos].type <= Cell.ANY): break
                            if (self.board[k][colPos].type == Cell.WHITE):
                                resRow = k
                                resCol = colPos
                                countWhite += 1
                        if (countWhite == 1 and cell.type == Cell.FORBIDDEN):
                            check = True
                            self.putSingleLamp(resRow, resCol)
                            self.setForbidden()
                            self.setAvailability()
                        if (countWhite == 0 and cell.type == Cell.WHITE):
                            check = True
                            self.putSingleLamp(rowPos, colPos)
                            self.setForbidden()
                            self.setAvailability()
            if not check: break

    def lastPreproc (self):
        for i, row in enumerate (self.board):
            for j, cell in enumerate (row):
                if cell.type == Cell.WHITE:
                    rowPos = i
                    colPos = j
                    check = False
                    # check rightside
                    for k in range (colPos+1, self.colSize):
                        if (k < 0 or k >= self.colSize or self.board[rowPos][k].type <= Cell.ANY): break
                        if (self.board[rowPos][k].type == Cell.WHITE):
                            check = True
                            break
                    # check leftside
                    for k in range (colPos-1, -1, -1):
                        if (k < 0 or k >= self.colSize or self.board[rowPos][k].type <= Cell.ANY): break
                        if (self.board[rowPos][k].type == Cell.WHITE):
                            check = True
                            break
                    # check downside
                    for k in range (rowPos+1, self.rowSize):
                        if (k < 0 or k >= self.rowSize or self.board[k][colPos].type <= Cell.ANY): break
                        if (self.board[k][colPos].type == Cell.WHITE):
                            check = True
                            break
                    # check upside
                    for k in range (rowPos-1, -1, -1):
                        if (k < 0 or k >= self.rowSize or self.board[k][colPos].type <= Cell.ANY): break
                        if (self.board[k][colPos].type == Cell.WHITE):
                            check = True
                            break
                    if not check: 
                        self.putSingleLamp(rowPos, colPos)
                        self.setAvailability()
    
    def putSingleLamp (self, rowPos, colPos):
        check = False
        if (rowPos < 0 or rowPos >= self.rowSize or colPos < 0 or colPos >= self.colSize): return
        if self.board[rowPos][colPos].type == Cell.LIT: self.board[rowPos][colPos].setType(Cell.COLLISION)
        elif self.board[rowPos][colPos].type == Cell.WHITE:
            self.board[rowPos][colPos].setType(Cell.LAMP)
            check = True
        if not check: return
        # lit rightside
        for i in range (colPos+1, self.colSize):
            if (i < 0 or i >= self.colSize): break
            if (self.board[rowPos][i].type <= Cell.ANY): break
            elif (self.board[rowPos][i].type == Cell.WHITE or self.board[rowPos][i].type == Cell.FORBIDDEN): self.board[rowPos][i].setType(Cell.LIT)
            elif self.board[rowPos][i].type == Cell.LAMP: self.board[rowPos][i].setType(Cell.COLLISION)
        # lit leftside
        for i in range (colPos-1, -1, -1):
            if (i < 0 or i >= self.colSize): break
            if (self.board[rowPos][i].type <= Cell.ANY): break
            elif (self.board[rowPos][i].type == Cell.WHITE or self.board[rowPos][i].type == Cell.FORBIDDEN): self.board[rowPos][i].setType(Cell.LIT)
            elif self.board[rowPos][i].type == Cell.LAMP: self.board[rowPos][i].setType(Cell.COLLISION)
        # lit downside
        for i in range (rowPos+1, self.rowSize):
            if (i < 0 or i >= self.rowSize): break
            if (self.board[i][colPos].type <= Cell.ANY): break
            elif (self.board[i][colPos].type == Cell.WHITE or self.board[i][colPos].type == Cell.FORBIDDEN): self.board[i][colPos].setType(Cell.LIT)
            elif self.board[i][colPos].type == Cell.LAMP: self.board[i][colPos].setType(Cell.COLLISION)
        # lit upside
        for i in range (rowPos-1, -1, -1):
            if (i < 0 or i >= self.rowSize): break
            if (self.board[i][colPos].type <= Cell.ANY): break
            elif (self.board[i][colPos].type == Cell.WHITE or self.board[i][colPos].type == Cell.FORBIDDEN): self.board[i][colPos].setType(Cell.LIT)
            elif self.board[i][colPos].type == Cell.LAMP: self.board[i][colPos].setType(Cell.COLLISION)
        

    def findBlackPosition (self):
        blackPosition = []
        for i, row in enumerate (self.board):
            for j, element in enumerate (row):
                if element.countAvailable < Cell.ANY and element.countAvailable > Cell.ZERO and element.checkAvailable() == True:
                    blackPosition.append([i, j])
        self.blackPosition = np.array(blackPosition)

    def findWhitePosition (self):
        whitePosition = []
        for i, row in enumerate (self.board):
            for j, element in enumerate (row):
                if element.type == Cell.WHITE:
                    check = True
                    for k in range (4):
                        nextRow = i + self.movement[0][k]
                        nextCol = j + self.movement[1][k]
                        if (nextRow >= 0 and nextCol >= 0 and nextRow < self.rowSize and nextCol < self.colSize):
                            if self.board[nextRow][nextCol].type <= Cell.FOUR: check = False
                    if check: whitePosition.append([i, j])
        self.whitePosition = np.array(whitePosition)

    def getRandomBlack (self):
        randomBlack = []
        for black in self.blackPosition:
            random = []
            rowPos = black[0]
            colPos = black[1]
            avail = self.board[rowPos][colPos].available
            if self.board[rowPos][colPos].countAvailable == Cell.ONE:
                for i in range (4): 
                    if avail[i] == Cell.AVAILABLE: random.append(i)
            elif self.board[rowPos][colPos].countAvailable == Cell.TWO:
                if avail[0] == Cell.AVAILABLE and avail[1] == Cell.AVAILABLE: random.append(0)
                if avail[1] == Cell.AVAILABLE and avail[2] == Cell.AVAILABLE: random.append(1)
                if avail[2] == Cell.AVAILABLE and avail[3] == Cell.AVAILABLE: random.append(2)
                if avail[0] == Cell.AVAILABLE and avail[3] == Cell.AVAILABLE: random.append(3)
                if avail[0] == Cell.AVAILABLE and avail[2] == Cell.AVAILABLE: random.append(4)
                if avail[1] == Cell.AVAILABLE and avail[3] == Cell.AVAILABLE: random.append(5)
            elif self.board[rowPos][colPos].countAvailable == Cell.THREE:
                if avail[0] == Cell.AVAILABLE and avail[1] == Cell.AVAILABLE and avail[2] == Cell.AVAILABLE: random.append(0)
                if avail[1] == Cell.AVAILABLE and avail[2] == Cell.AVAILABLE and avail[3] == Cell.AVAILABLE: random.append(0)
                if avail[0] == Cell.AVAILABLE and avail[2] == Cell.AVAILABLE and avail[3] == Cell.AVAILABLE: random.append(0)
                if avail[0] == Cell.AVAILABLE and avail[1] == Cell.AVAILABLE and avail[3] == Cell.AVAILABLE: random.append(0)
            elif self.board[rowPos][colPos].countAvailable == Cell.FOUR: random.append(0)
            randomBlack.append(random)
        return randomBlack
  
    def putBlackLamps (self, position):
        self.board = copy.deepcopy(self.original) # copy the original board back to board
        for i, black in enumerate (self.blackPosition): # for every black squares in blackPosition
            rowPos = black[0] # row position (int)
            colPos = black[1] # col position (int)
            element = self.board[rowPos][colPos].countAvailable # calculate the number of possible bulb placement around black
            if element == Cell.ONE:
                if position[i] == 0: self.putSingleLamp(rowPos, colPos+1)
                elif position[i] == 1: self.putSingleLamp(rowPos+1, colPos)
                elif position[i] == 2: self.putSingleLamp(rowPos, colPos-1)
                elif position[i] == 3: self.putSingleLamp(rowPos-1, colPos)
            elif element == Cell.TWO:
                if position[i] == 0:
                    self.putSingleLamp(rowPos, colPos+1)
                    self.putSingleLamp(rowPos+1, colPos)
                elif position[i] == 1:
                    self.putSingleLamp(rowPos+1, colPos)
                    self.putSingleLamp(rowPos, colPos-1)
                elif position[i] == 2:
                    self.putSingleLamp(rowPos, colPos-1)
                    self.putSingleLamp(rowPos-1, colPos)
                elif position[i] == 3:
                    self.putSingleLamp(rowPos-1, colPos)
                    self.putSingleLamp(rowPos, colPos+1)
                elif position[i] == 4:
                    self.putSingleLamp(rowPos, colPos+1)
                    self.putSingleLamp(rowPos, colPos-1)
                elif position[i] == 5:
                    self.putSingleLamp(rowPos+1, colPos)
                    self.putSingleLamp(rowPos-1, colPos)
            elif element == Cell.THREE:
                if position[i] == 0:
                    self.putSingleLamp(rowPos, colPos+1)
                    self.putSingleLamp(rowPos+1, colPos)
                    self.putSingleLamp(rowPos, colPos-1)
                elif position[i] == 1:
                    self.putSingleLamp(rowPos+1, colPos)
                    self.putSingleLamp(rowPos, colPos-1)
                    self.putSingleLamp(rowPos-1, colPos)
                elif position[i] == 2:
                    self.putSingleLamp(rowPos, colPos-1)
                    self.putSingleLamp(rowPos-1, colPos)
                    self.putSingleLamp(rowPos, colPos+1)
                elif position[i] == 3:
                    self.putSingleLamp(rowPos-1, colPos)
                    self.putSingleLamp(rowPos, colPos+1)
                    self.putSingleLamp(rowPos+1, colPos)
            elif element == Cell.FOUR:
                self.putSingleLamp(rowPos, colPos+1)
                self.putSingleLamp(rowPos+1, colPos)
                self.putSingleLamp(rowPos, colPos-1)
                self.putSingleLamp(rowPos-1, colPos)

    def putWhiteLamps (self, position):
        for i, white in enumerate (self.whitePosition):
            rowPos = white[0]
            colPos = white[1]
            if (position[i] == 1):
                self.putSingleLamp(rowPos, colPos)

    def saveBoard (self):
        self.original = copy.deepcopy(self.board)

    def updateFitness (self, punishment):
        collision = 0
        whiteTiles = 0
        blackLamp = 0
        centerWhite = 0
        endBlack = 0
        fitness = 0
        trapped = 0
        for i, row in enumerate (self.board):
            for j, element in enumerate (row):
                if element.type == Cell.WHITE or element.type == Cell.FORBIDDEN:
                    whiteTiles+=1
                    # find center white
                    check = True
                    for k in range (4):
                        nextRow = i + self.movement[0][k]
                        nextCol = j + self.movement[1][k]
                        if (nextRow >= 0 and nextCol >= 0 and nextRow < self.rowSize and nextCol < self.colSize):
                            if (self.board[nextRow][nextCol].type > Cell.ANY and self.board[nextRow][nextCol].type <= Cell.FORBIDDEN):
                                check = False
                                break
                    if check: centerWhite+=1
                    if element.type == Cell.FORBIDDEN:
                        rowPos = i
                        colPos = j
                        check = False
                        # check rightside
                        for k in range (colPos+1, self.colSize):
                            if (k < 0 or k >= self.colSize or self.board[rowPos][k].type <= Cell.ANY): break
                            if (self.board[rowPos][k].type == Cell.WHITE):
                                check = True
                                break
                        # check leftside
                        for k in range (colPos-1, -1, -1):
                            if (k < 0 or k >= self.colSize or self.board[rowPos][k].type <= Cell.ANY): break
                            if (self.board[rowPos][k].type == Cell.WHITE):
                                check = True
                                break
                        # check downside
                        for k in range (rowPos+1, self.rowSize):
                            if (k < 0 or k >= self.rowSize or self.board[k][colPos].type <= Cell.ANY): break
                            if (self.board[k][colPos].type == Cell.WHITE):
                                check = True
                                break
                        # check upside
                        for k in range (rowPos-1, -1, -1):
                            if (k < 0 or k >= self.rowSize or self.board[k][colPos].type <= Cell.ANY): break
                            if (self.board[k][colPos].type == Cell.WHITE):
                                check = True
                                break
                        if not check: trapped+=1
                elif element.type == Cell.COLLISION: collision+=1
                elif element.type <= Cell.FOUR:
                    if element.checkAvailable(): 
                        blackLamp+=1
                        # cek ujung
                        if ((i == 0 and (j == self.colSize-1 or j == 0)) or (i == self.rowSize-1 and (j == self.colSize-1 or j == 0))):
                            if element.type == Cell.TWO: endBlack+=1
                        # cek tepi
                        if (((i == 0 or i == self.rowSize-1) and (j < self.colSize-1 and j > 0)) or ((j == 0 or j == self.colSize-1) and (i < self.rowSize-1 and i > 0))):
                            if element.type == Cell.THREE: endBlack+=1
                
        fitness += (collision * punishment[0] + blackLamp * punishment[1] + whiteTiles * punishment[2] + endBlack * punishment[3] + centerWhite * punishment[4] + trapped * punishment[5])
        violation = [collision, blackLamp, whiteTiles, centerWhite, endBlack, trapped]
        return fitness, violation

In [4]:
class Wolf:
    def __init__(self, board) -> None:
        self.board = board
    
    def updateBlackPosition (self, newPosition):
        self.board.putBlackLamps(newPosition)
        self.blackPosition = newPosition
        self.board.setAvailability()

    def updateWhitePosition (self, newPosition):
        self.board.putWhiteLamps(newPosition)
        self.whitePosition = newPosition

    def updateFitness (self, punishment):
        self.fitness, self.violation = self.board.updateFitness(punishment)
        return self.fitness

In [ ]:
import numpy as np
import random as rand
import copy

class Game:
    def __init__(self, seed, population, punishment, epoch, rank, preproc) -> None:
        rand.seed(seed)
        np.random.seed(seed)
        self.population = population
        self.punishment = punishment
        self.epoch = epoch
        self.rank = rank
        self.preproc = preproc

    def play (self, playBoard, tiles, actions):
        self.board = Board(playBoard)
        self.board.setForbidden()
        self.board.setAvailability()
        self.tempBoard = copy.deepcopy(self.board).board
        fitness = 0
        if self.preproc: 
            self.board.preproc()
            self.board.setForbidden()
            self.board.setAvailability()
            self.board.preprocSecond()
            self.board.setForbidden()
            self.board.setAvailability()
            fitness = self.board.updateFitness(self.punishment)[0]
            self.clickBoard(tiles, actions, self.board.board, fitness)
        if fitness == 0: return
        firstResult = self.firstGWO()
        max = 2**31 - 1
        ans = None
        for wolf in firstResult:
            if self.preproc:
                wolf.board.preprocSecond()
                wolf.board.setForbidden()
                wolf.board.setAvailability()
                fitness = wolf.board.updateFitness(self.punishment)[0]
                self.clickBoard(tiles, actions, wolf.board.board, fitness)
                if fitness == 0: return
            res = self.secondGWO(wolf)
            if self.preproc:
                res.board.lastPreproc()
                res.updateFitness(self.punishment)
            fitness = res.fitness
            self.clickBoard(tiles, actions, res.board.board, fitness)
            if fitness == 0: return
            if res.fitness < max:
                max = res.fitness
                ans = res
            if max == 0: 
                break
        self.clickBoard(tiles, actions, ans.board.board, fitness)

    def experiment (self, firstBoard):
        self.board = Board(firstBoard)
        self.board.setForbidden()
        self.board.setAvailability()
        ans = None
        if self.preproc: 
            self.board.preproc()
            self.board.setForbidden()
            self.board.setAvailability()
            self.board.preprocSecond()
            self.board.setForbidden()
            self.board.setAvailability()
            ans = Wolf(self.board)
            if ans.updateFitness(self.punishment) == 0: return ans
        firstResult = self.firstGWO()
        max = 2**31 - 1
        for wolf in firstResult:
            if self.preproc:
                wolf.board.preprocSecond()
                wolf.board.setForbidden()
                wolf.board.setAvailability()
                if wolf.updateFitness(self.punishment) == 0: return wolf
            res = self.secondGWO(wolf)
            if self.preproc:
                res.board.lastPreproc()
                res.updateFitness(self.punishment)
            if res.fitness == 0: return res
            if res.fitness < max:
                max = res.fitness
                ans = copy.deepcopy(res)
        return ans

    def firstGWO (self):
        population = []
        self.board.findBlackPosition()
        randomBlack = self.board.getRandomBlack()
        if len(randomBlack) == 0: return [Wolf(self.board)]
        max = 2**31 - 1
        best = ''
        for _ in range (self.population): population.append(Wolf(copy.deepcopy(self.board)))
        for wolf in population:
            initPosition = []
            for r in randomBlack:
                initPosition.append(rand.choice(r))
            initPosition = np.array(initPosition)
            wolf.board.saveBoard()
            wolf.updateBlackPosition(initPosition)
            wolf.updateFitness(self.punishment)
            if wolf.fitness < max:
                max = wolf.fitness
                best = copy.deepcopy(wolf)
        population.sort(key = lambda wolf : wolf.fitness)
        alpha = population[0]
        beta = population[1]
        delta = population[2]
        a = 2
        for i in range (self.epoch):
            for wolf in population:
                a = 2 - (i/self.epoch)*2
                thisPosition = wolf.blackPosition
                dimension = len(randomBlack)
                A1 = 2 * a * np.random.sample(size=dimension) - a
                A2 = 2 * a * np.random.sample(size=dimension) - a
                A3 = 2 * a * np.random.sample(size=dimension) - a
                C1 = 2 * np.random.sample(size=dimension) 
                C2 = 2 * np.random.sample(size=dimension) 
                C3 = 2 * np.random.sample(size=dimension) 
                X1 = alpha.blackPosition - A1 * abs(C1 * alpha.blackPosition - thisPosition)
                X2 = beta.blackPosition - A2 * abs(C2 * beta.blackPosition - thisPosition)
                X3 = delta.blackPosition - A3 * abs(C3 * delta.blackPosition - thisPosition)
                newPosition = (X1 + X2 + X3) / 3.0
                newPosition = np.where(newPosition < 0, -newPosition, newPosition)
                randomBlackSizes = []
                for r in randomBlack: randomBlackSizes.append(len(r))
                newPosition = np.round(newPosition).astype(int)
                newPosition %= randomBlackSizes
                for i, r in enumerate (randomBlack):
                    newPosition[i] = r[newPosition[i]]
                wolf.updateBlackPosition(newPosition)
                wolf.updateFitness(self.punishment)
                if wolf.fitness < max:
                    max = wolf.fitness
                    best = copy.deepcopy(wolf)
            population.sort(key = lambda wolf : wolf.fitness)
            alpha = population[0]
            beta = population[1]
            delta = population[2]
        result = self.roulette(population)
        result.insert(0, best)
        return result
    
    def secondGWO (self, wolf):
        firstWolf = copy.deepcopy(wolf)
        population = []
        max = 2**31 - 1
        best = ''
        firstWolf.board.findWhitePosition()
        for _ in range (self.population): population.append(copy.deepcopy(firstWolf))
        for wolf in population:
            initPosition = np.random.randint(2, size=len(firstWolf.board.whitePosition))
            wolf.board.saveBoard()
            wolf.updateWhitePosition(initPosition)
            wolf.updateFitness(self.punishment)
            if wolf.fitness < max:
                if wolf.fitness == 0: return wolf
                max = wolf.fitness
                best = copy.deepcopy(wolf)
        population.sort(key = lambda wolf : wolf.fitness)
        alpha = population[0]
        beta = population[1]
        delta = population[2]
        a = 2
        for i in range (self.epoch):
            for wolf in population:
                a = 2 - (i/self.epoch)*2
                thisPosition = wolf.whitePosition
                dimension = len(thisPosition)
                A1 = 2 * a * np.random.sample(size=dimension) - a
                A2 = 2 * a * np.random.sample(size=dimension) - a
                A3 = 2 * a * np.random.sample(size=dimension) - a
                C1 = 2 * np.random.sample(size=dimension) 
                C2 = 2 * np.random.sample(size=dimension) 
                C3 = 2 * np.random.sample(size=dimension) 
                X1 = alpha.whitePosition - A1 * abs(C1 * alpha.whitePosition - thisPosition)
                X2 = beta.whitePosition - A2 * abs(C2 * beta.whitePosition - thisPosition)
                X3 = delta.whitePosition - A3 * abs(C3 * delta.whitePosition - thisPosition)
                newPosition = (X1 + X2 + X3) / 3.0
                newPosition = np.where(newPosition < 0, -newPosition, newPosition)
                newPosition = np.round(newPosition).astype(int)
                newPosition %= 2
                wolf.updateWhitePosition(initPosition)
                wolf.updateFitness(self.punishment)
                if wolf.fitness < max:
                    if wolf.fitness == 0: return wolf
                    max = wolf.fitness
                    best = copy.deepcopy(wolf)
            population.sort(key = lambda wolf : wolf.fitness)
            alpha = population[0]
            beta = population[1]
            delta = population[2]
        return best
    
    def roulette (self, population): # Involves roulette wheel with rank selection
        result = []
        size = int (self.population*self.rank)
        for i in range (size):
            probabilities = []
            totalFitness = sum(wolf.fitness for wolf in population)
            for wolf in population:
                probabilities.append(wolf.fitness/totalFitness)
            spin = rand.uniform(0,1)
            accumulatedProb = 0
            for i, prob in enumerate(probabilities):
                accumulatedProb += prob
                if accumulatedProb >= spin:
                    if population[i].violation[0] == 0 and population[i].violation[1] == 0 and population[i].violation[3] == 0 and population[i].violation[4] == 0 and population[i].violation[5] == 0:
                        result.append(population[i])
                    population.pop(i)
                    break
        return result

    def clickBoard (self, tiles, actions, board, fitness):
        i = 1
        for rowTemp, rowBoard in zip(self.tempBoard, board):
            for cellTemp, cellBoard in zip(rowTemp, rowBoard):
                if (cellTemp.type == Cell.LAMP and cellBoard.type != Cell.LAMP) or (cellTemp.type != Cell.LAMP and cellBoard.type == Cell.LAMP):
                    tiles[i].click()
                if (fitness > 0):
                    if (cellTemp.type == Cell.FORBIDDEN and cellBoard.type != Cell.FORBIDDEN) or (cellTemp.type != Cell.FORBIDDEN and cellBoard.type == Cell.FORBIDDEN):
                        actions.context_click(tiles[i]).perform()
                i+=1
        self.tempBoard = copy.deepcopy(board)

In [ ]:
import pickle as pc
import datetime
import sys
from tabulate import tabulate
import statistics

if __name__ == "__main__":
    # get current date and time
    now = datetime.datetime.now()

    # format date and time as a string
    date_string = now.strftime("%Y-%m-%d-%H-%M-%S")

    # create output file with date and time in the name
    filename = f"Log/output_{date_string}.txt"

    # Open the file in write mode
    #log_file = open(filename, 'w')

    # Redirect the standard output to the file
    #sys.stdout = log_file
    dataset = ['7_easy', '7_normal', '7_hard',
               '10_easy', '10_normal', '10_hard',
               '14_easy', '14_normal', '14_hard',
               '25_easy', '25_normal', '25_hard',
               '30_daily', '30_weekly', '40_monthly']
    seed = 180820
    population = [200]
    base = 1
    punishment = [5*base, 20*base, 1*base, 100*base, 100*base, 100*base]
    epoch = [500]
    rank = [0.1]
    preproc = [True, False]
    boardCount = 100
    datasetToTest = [dataset[0],dataset[1],dataset[2]]
    headers = ['Preprocessing', 'Tingkat kesulitan', 'Nomor papan', 'Nilai fitness akhir', 'Pelanggaran']
    final_result = []
    
    fitness = []
    filename = f"output_{date_string}"
    with open(f"ResultEXP/{filename}.txt", 'a') as f:
        for d in datasetToTest:
            for pop in population:
                for e in epoch:
                    for r in rank:
                        for prep in preproc:
                            jumlahPapanSelesai = 0
                            for i in range (boardCount):
                                print (f"mencoba tingkat kesulitan {d} papan ke-{i}")
                                inventory = pc.load(open(f"Dataset/{d}.pkl", 'rb'))
                                firstBoard = inventory[i]
                                game = Game(seed, pop, punishment, e, r, prep)
                                ans = game.experiment(firstBoard)
                                violation = ','.join(map(str, ans.violation))
                                result = [preproc, d, i, ans.fitness, violation]
                                final_result.append(result)
                                fitness.append(ans.fitness)
                                if ans.fitness == 0: jumlahPapanSelesai+=1
                                ans.board.printBoard()
                                print(ans.fitness, ans.violation)
                            avg = sum(fitness) / boardCount
                            std_dev = statistics.stdev(fitness)
                            f.write(f"{preproc}-{d}-{punishment}-{pop}-{e}-{r}-{prep}: avg = {avg} | std = {std_dev} | selesai = {jumlahPapanSelesai}/{boardCount}\n")
    f.close()
    print (f'{jumlahPapanSelesai} dari {boardCount} papan selesai')
    print (f'persentase keberhasilan sebesar {jumlahPapanSelesai/boardCount} persen')
    table = tabulate(final_result, headers=headers, tablefmt='latex')
    filename = f"output_{date_string}"
    with open(f"ResultTEX/{filename}.tex", 'w') as f:
        f.write(table)